In [1]:
import pandas as pd
from math import log2 

import import_ipynb
from LogicLanguage import LogicLanguage
from BoolRepres import BoolRepres

class ConstraintDefiner():
    
    def __init__(self, nameVars, maxValues): 
        # Init: vars + logicClass  # priorityList -> [(var1, var2),(var2,var4),...]=> var1 ha priorità su var2 nel SAT ecc.
        self.constr   = [] 
        self.dfVars   = self.__init_dfVars(nameVars, maxValues)
        self.df_default = None
        
        self.edgeList = [] # list of edges. es: [(a,b), (c,b), (b,k), ...]# Assume No cilci: [(a,b), (b,a)] NO!!!
        self.oper = self.__init_operators(None)
        self.mll  = LogicLanguage()  

        
    def __init_operators(self, oper):
        if oper is None:
            oper = {"AND":'and', 'OR':'or', 'NOT':'not', 'TRUE':'true', 'FALSE':'false'} 
            #print('Will use the following operator as default:')
            #print( ''.join([f'{k} as {v}, ' for k,v in oper.items()])[:-1]) 
            return oper
        return oper

    #=============================================================================
    # ------------------------- INITIALIZATION VARIABLES -------------------------
    
    def __init_dfVars(self, nameVars, maxValues):
        # Rappresentazione in bit
        dfVars = (pd.DataFrame({'V':nameVars, 'maxV':maxValues})
                      .assign(nbit=lambda x: [ 1 if mV==0 else int(log2(mV))+1 for mV in x.maxV])
                      .set_index('V'))
        for col in ['XL','XU','YL','YU']:
            dfVars[col] = dfVars.apply(lambda v: BoolRepres(v.name+'_'+col, v['nbit']) , axis=1)
        return dfVars
    
    def get_varsName(self):
        return self.dfVars[['XL','XU','YL','YU']].applymap(lambda br: br.bitsName).sum().sum()

    #=============================================================================
    # -------------------------- SET USER OPTIONS SETTING ------------------------
    def set_saveSpace(self, saveSpace=False):
        self.mll.set_saveSpace(saveSpace)
        
    def set_seed(self, seed):
        self.mll.set_seed(seed)
        
    def set_resetTree(self, resetTree):
        self.mll.set_resetTree(resetTree)
        
    def set_childrenChoice(self, _type='random'):
        # default -> 'random'
        # 'min_avgLenPath' or 'min_power': 
        self.mll.set_childrenChoice(_type=_type)
    
    def set_saturate_freeVars(self, saturate=None): # self, defaultVars, groups=None 
        
        if saturate is None:
            self.mll.set_groups(None)
            self.mll.set_default(None)
            self.mll.set_saturate_options(False)
            return
               
        if saturate not in ['all','effective']:
            print('Wrong paramiter insered')
            return 
        
        # if here, for sure saturate = 'all'
        # Create default dictionary
        # Assumo default-> Lower bound = 0, Upper bound = max
        def_list, defaultVal, groups = [], {}, None
        def_list += self.dfVars['XL'].apply(lambda br: br.set_to(0).to_dict()).tolist()
        def_list += self.dfVars['YL'].apply(lambda br: br.set_to(0).to_dict()).tolist()
        def_list += self.dfVars[['maxV','XU']].apply(lambda row: row['XU'].set_to(row['maxV']).to_dict(), axis=1).tolist()
        def_list += self.dfVars[['maxV','YU']].apply(lambda row: row['YU'].set_to(row['maxV']).to_dict(), axis=1).tolist()
        for d in def_list: 
            defaultVal |= d
        
        # Create atomsGroups, if required 
        if saturate == 'effective':
            groups = self.dfVars[['XL','XU','YL','YU']].applymap(lambda br: br.get_bitsName()).values.reshape(len(self.dfVars)*4)
        
        # set LogicLanguage
        self.mll.set_groups(groups)
        self.mll.set_default(defaultVal)
        self.mll.set_saturate_options(state = True, freeVars=saturate, values='default')
        
        #return defaultVal
    
    #-------------------------------------------------------------------------------
    def set_priority(self, priorityList=[] , digitPrior='None'): 
        # priorityList -> list of edges. es: [(a,b), (c,b), (b,k), ...]# Assume No cilci: [(a,b), (b,a)] NO!!!
        # priorityList -> work in progress
        
        if  digitPrior == 'None' : 
            self.edgeList = [] 
            
        elif digitPrior == 'Simple': 
            # Ogni bit di una variabile var_XL/XU/YL/YU_k ha priorità sul relavito bit k-1
            # es a_XL_3 ha priorità su a_XL_2,  a_XU_3 ha priorità su a_XU_2, ecc.
            self.edgeList = self.dfVars[['XL','XU','YL','YU']].applymap(lambda br: [(br[i],br[i+1]) for i in range(br.nBits-1)]).sum().sum()
            
        elif digitPrior == 'Complete':
            # Ogni variabile var_XL/XU/YL/YU_k ha priorità su tutti i bit k-1. 
            # es: a_XL_3 ha priorità su a_XL_2, a_XU_2, a_YL_2, a_YU_2
            temp = [] 
            for (var, nBit) in self.dfVars['nbit'].reset_index().values:
                for n in range(nBit):
                    for iSide in ['XL','XU','YL','YU']:
                        for jSide in ['XL','XU','YL','YU']:
                            temp.append( (f"{var}_{iSide}_{nBit-n}", f"{var}_{jSide}_{nBit-n-1}") )
            self.edgeList = temp
        else: 
            print(f"Paramiter {digitPrior} not finded")
            return None
                  
        # Define user priority
        temp = [] 
        for v,u in priorityList:
            for v_side in ['XL','XU','YL','YU']:
                for u_side in ['XL','XU','YL','YU']:
                    temp.append( (f"{v}_{v_side}_0", f"{u}_{u_side}_{self.dfVars.at[u,'nbit']-1}") )
        self.edgeList += temp
        
        # update priority in LogicLanguage
        self.mll.set_priority(self.edgeList)
                    

       
    #============================================================================= 
    # --------------------------- SAT SOLVER ----------------------------
    def reject_solut(self, solutSet):
        sol=[]
        for k,v in solutSet.items():
            if   v == "true"  : sol.append(k)
            elif v == "false" : sol.append(f'not {k}')
            else: print("wrong paramiter: ignored. Only True or False ")
        return f'not({" and ".join(sol)})'
         
    #-------------------------------------------------------------------        
    def addConstr(self, logicStr): 
        self.constr.append(f'({logicStr})')   
        self.mll.add_constr(f'({logicStr})', reset_old=False)
  
    #-------------------------------------------------------------------  
    def Sat_Solver(self, verbose=False, test=None):
        return self.mll.sat_Solver(verbose=verbose, test=test)#, self.mll.all_varTree # ritorno treeVars x grafico albero delle variabili
    
    #=============================================================================
    # --------------------------- INITIAL CONSTRAINTS ----------------------------
    def save_Constr_txt(self, fileName='saved_Constraints'):
        with open(fileName+'.txt', 'w') as f:
            f.write('\n'.join(self.constr)) 
    #-------------------------------------------------------------------            
    def load_Constr_txt(self, fileName='saved_Constraints', returns=False):
        with open(fileName+'.txt', 'r') as f:
            self.constr = f.read().split('\n')
            if returns: return self.constr 
        
    #------------------------------------------------------------------- 
    def init_constr(self):
        formula, dfVars = [], self.dfVars  
        #---------------------------------
        # Constraints:  Phy_init_max --> Var_X/Y L/U <= maxVal 
        for col in ['XL','XU','YL','YU']:
            formula += dfVars.apply(lambda x: x[col] <= x['maxV'], axis=1).tolist()
            
        #---------------------------------
        # Constraints: Phy_init_interval --> Var_X L <= Var_X L  
        formula += dfVars.apply(lambda x: x['XL'] <= x['XU'], axis=1).tolist()
        formula += dfVars.apply(lambda x: x['YL'] <= x['YU'], axis=1).tolist()
        
        #---------------------------------
        # Constraints: Phy_init_rule --> define the equivalent to " a_X -> not a_y" ==> "not a_x or a_y" in intervals 
        def init_rule(row, iL, iU, jL, jU):
            v_x = f"{row[iL] > 0} or {row[iU] < row['maxV']}"
            v_y = f"{row[jL] == 0} and {(row[jU] == row['maxV'])}"
            return f'(not({v_x}) or ({v_y}))'
        
        formula += dfVars.apply(lambda row: init_rule(row,'XL','XU','YL','YU') , axis=1).tolist() 
        formula += dfVars.apply(lambda row: init_rule(row,'YL','YU','XL','XU') , axis=1).tolist() 
        
        or_X = dfVars.apply(lambda row: f"{row['XL']>0} or {row['XU']<row['maxV']}" , axis=1).tolist()
        or_Y = dfVars.apply(lambda row: f"{row['YL']>0} or {row['YU']<row['maxV']}" , axis=1).tolist()
        return f"({' and '.join(formula)}) and ({' or '.join(or_X)}) and ({' or '.join(or_Y)})"
    
    #=============================================================================
    # ------------------------------- DECODE RULE -------------------------------    
    def decodeRule(self, res): #res = solutSet = {nomeVar_[X|Y]_[L|U]_nbit: true/false}
        #------------------------
        # Decode Bool:
        def decode(varBits, res):
            return int( ''.join(['1' if res[k]=='true' else '0' for k in varBits]), 2)  
        df_res = self.dfVars[['XL','XU','YL','YU']].applymap(lambda br: decode(br.bits, res))
        
        #-----------------------
        # Rule extraction
        notX = (df_res.XL==0) & (df_res.XU==self.dfVars.maxV) # Da rimuovere da X
        notY = (df_res.YL==0) & (df_res.YU==self.dfVars.maxV) # Da rimuovere da Y

        #-----------------------
        # type Output: only Dataset 
        df_X = df_res.loc[-notX,:][['XL','XU']].rename(columns={'XL':'L','XU':'U'}).astype(int)
        df_Y = df_res.loc[-notY,:][['YL','YU']].rename(columns={'YL':'L','YU':'U'}).astype(int)
        
        # TEMP: Pretty Output -> String intervals 
        df_X_pretty = df_X.index+'['+ df_X.L.astype(str)+','+df_X.U.astype(str)+']'
        df_Y_pretty = df_Y.index+'['+ df_Y.L.astype(str)+','+df_Y.U.astype(str)+']'
        s = f"{', '.join(df_X_pretty.tolist())} --> {', '.join(df_Y_pretty.tolist())}"
        
        return df_X, df_Y, s # in form: {'df_X':df_X, 'df_Y':df_Y, 'str':s}
        
    
    #=============================================================================
    #----------------------------- COMMON FUNCTIONS ------------------------------
    def __extend(self, var, sL, sU, L, U):
        return f"({var[sL] <= L} and {U <= var[sU]})"
    
    def __reduce(self, var, sL, sU, L, U):
        return f"({L <= var[sL]} and {var[sU] <= U})"
    
    def __notIn(self, var, side):
        return f"({var[side+'L']==0} and {var[side+'U']==var['maxV']})"
    
    def __nsupp(self, var, L, U):
        return '('+self.__reduce(var,'XL','XU',L,U)+' or '+ self.__reduce(var,'YL','YU',L,U)+')'
    
    # --------------------------- NOT SUPPORTED RULES ----------------------------
    def NotSupported(self, res):      
        formula, dfVars = [], self.dfVars
        X, Y, s = self.decodeRule(res)
       
        formula += X.apply(lambda row: self.__nsupp(dfVars.loc[row.name],row.L,row.U), axis=1).tolist()
        formula += Y.apply(lambda row: self.__nsupp(dfVars.loc[row.name],row.L,row.U), axis=1).tolist() 
        return f"(not({' and '.join(formula)}))"

    # --------------------------- NOT CONFIDENT RULES ----------------------------   
    def NotConfident(self, res):
        formula, dfVars = [], self.dfVars
        X, Y, s = self.decodeRule(res)
      
        formula += dfVars.drop(X.index).apply(lambda row: self.__notIn(row,'X'), axis=1).tolist()  
        formula += X.apply(lambda row: self.__extend(dfVars.loc[row.name],'XL','XU',row.L,row.U), axis=1).tolist()        
        formula += Y.apply(lambda row: self.__reduce(dfVars.loc[row.name],'YL','YU',row.L,row.U), axis=1).tolist()
        return f"(not({' and '.join(formula)}))"

    # ----------------------------- SUPPORTED RULES ------------------------------
    def Confident(self, res):
        formula, dfVars = [], self.dfVars
        X, Y, s = self.decodeRule(res)
        
        formula += dfVars.drop(Y.index).apply(lambda row: self.__notIn(row,'Y'), axis=1).tolist()  
        formula += Y.apply(lambda row: self.__extend(dfVars.loc[row.name],'YL','YU', row.L, row.U), axis=1).tolist()
        formula += X.apply(lambda row: self.__reduce(dfVars.loc[row.name],'XL','XU', row.L, row.U), axis=1).tolist()
        return f"(not({' and '.join(formula)}))"
    

importing Jupyter notebook from LogicLanguage.ipynb
importing Jupyter notebook from BoolRepres.ipynb
